# Day 22

## Read input

In [134]:
import re
from collections import namedtuple

from utils import read_input

Point = namedtuple('Point', ['x', 'y', 'z'])
Instruction = namedtuple('Instruction', ['action', 'start', 'end'])

def transformer(line):
    m = re.match(r'(?P<action>on|off) x=(?P<x>-?\d+..-?\d+),y=(?P<y>-?\d+..-?\d+),z=(?P<z>-?\d+..-?\d+)', line)
    action = m.group('action')
    x_range = m.group('x')
    y_range = m.group('y')
    z_range = m.group('z')
    
    x1, x2 = x_range.split('..')
    y1, y2 = y_range.split('..')
    z1, z2 = z_range.split('..')
    
    start = Point(x=int(x1), y=int(y1), z=int(z1))
    end = Point(x=int(x2), y=int(y2), z=int(z2))
    
    return Instruction(action, start, end)
    
instructions = read_input(22, transformer)

## Part 1

In [135]:
def get_cuboids_in_range(start, end):
    cuboids = []
    if start.x < -50 and end.x < -50 or start.x > 50 and end.x > 50:
        return []
    if start.y < -50 and end.y < -50 or start.y > 50 and end.y > 50:
        return []
    if start.z < -50 and end.z < - 50 or start.z > 50 and end.z > 50:
        return []

    min_x = max(-50, min(50, start.x))
    max_x = min(50, max(-50, end.x))
    min_y = max(-50, min(50, start.y))
    max_y = min(50, max(-50, end.y))
    min_z = max(-50, min(50, start.z))
    max_z = min(50, max(-50, end.z))
    
    
    for x in range(min_x, max_x + 1):
        for y in range(min_y, max_y + 1):
            for z in range(min_z, max_z + 1):
                cuboids.append(Point(x,y,z))
    return cuboids

I learned today that `set` has two methods for removing items: `discard` and `remove`. I've been using `remove` with an `if` guard all these years but there's a nice difference:

> discard(...) method of builtins.set instance
>    Remove an element from a set if it is a member.
>
>    If the element is not a member, do nothing.

>remove(...) method of builtins.set instance
>    Remove an element from a set; it must be a member.
>
>    If the element is not a member, raise a KeyError.

So whenever you need to remove items that might not exist in a set, `discard` is a good option.

In [136]:
def activate_cuboids(instructions):
    lit = set()

    for instruction in instructions:
        for point in get_cuboids_in_range(instruction.start, instruction.end):
            match instruction.action:
                case 'on':
                    lit.add(point)
                case 'off':
                    lit.discard(point)
    return lit

In [137]:
result = len(activate_cuboids(instructions))
print(f'Solution: {result}')
assert result == 648681

Solution: 648681


## Part 2

In [138]:
def crop(one, another):
    new = {
        'start': [],
        'end': []
    }
    for axis in ['x', 'y', 'z']:
        one_start = getattr(one.start, axis)
        one_end = getattr(one.end, axis)
        another_start = getattr(another.start, axis)
        another_end = getattr(another.end, axis)
        
        max_start = max(one_start, another_start)
        min_end = min(one_end, another_end)
        
        if max_start < min_end + 1:
            new_start = max_start
            new_end = min_end
        else:
            new_start = 0
            new_end = 0
        new['start'].append(new_start)
        new['end'].append(new_end)

    start = Point(*new['start'])
    end = Point(*new['end'])
    return Area(start, end)

def empty(area):
    xs = (area.start.x, area.end.x)
    ys = (area.start.y, area.end.y)
    zs = (area.start.z, area.end.z)
    return xs == (0, 0) or ys == (0, 0) or zs == (0, 0)
    
class Area:
    
    def __init__(self, start, end):
        self.start = start
        self.end = end
        self.off = []

    def subtract(self, other):
        cropped = crop(other, self)
        if empty(cropped):
            return
        for other in self.off:
            other.subtract(cropped)
        self.off.append(cropped)
            
    def volume(self):
        full_volume = (self.end.x - self.start.x + 1) * (self.end.y - self.start.y + 1) * (self.end.z - self.start.z + 1)
        off_volume = sum(area.volume() for area in self.off)        
        return full_volume - off_volume
    
    def __repr__(self):
        return f'<Area>: {self.start}, {self.end}'

In [140]:
areas = []

for instruction in instructions:
    area = Area(instruction.start, instruction.end)
    
    for other in areas:
        other.subtract(area)
        
    if instruction.action == 'on':
        areas.append(area)
        
result = sum(area.volume() for area in areas)
print(f'Solution: {result}')
assert result == 1302784472088899

Solution: 1302784472088899
